In [1]:
%matplotlib inline
from matplotlib.pyplot import figure, show
import matplotlib.pyplot as plt
import json
import csv
import pandas as pd
import sklearn.feature_extraction.text as sk_text
import io
import requests
import numpy as np
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import metrics

In [2]:
#------------------PROFESSOR'S CODE, HIDDEN GEM--------------------------
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) * (normalized_high - normalized_low) + normalized_low

In [3]:
df = pd.read_csv('census_data/DEC_00_110S_DP4_with_ann.csv', encoding="utf-8")
df.head(20)

,GEO.id,GEO.id2,GEO.display-label,HC01_VC01,HC02_VC01,HC01_VC03,HC02_VC03,HC01_VC04,HC02_VC04,HC01_VC05,...,HC01_VC109,HC02_VC109,HC01_VC110,HC02_VC110,HC01_VC111,HC02_VC111,HC01_VC112,HC02_VC112,HC01_VC113,HC02_VC113
0,Id,Id2,Geography,Number; Total housing units,Percent; Total housing units,Number; Total housing units - UNITS IN STRUCTU...,Percent; Total housing units - UNITS IN STRUCT...,Number; Total housing units - UNITS IN STRUCTU...,Percent; Total housing units - UNITS IN STRUCT...,Number; Total housing units - UNITS IN STRUCTU...,...,Number; Specified renter-occupied units - GROS...,Percent; Specified renter-occupied units - GRO...,Number; Specified renter-occupied units - GROS...,Percent; Specified renter-occupied units - GRO...,Number; Specified renter-occupied units - GROS...,Percent; Specified renter-occupied units - GRO...,Number; Specified renter-occupied units - GROS...,Percent; Specified renter-occupied units - GRO...,Number; Specified renter-occupied units - GROS...,Percent; Specified renter-occupied units - GRO...
1,5001000US0101,0101,"Congressional District 1 (110th Congress), Ala...",281766,100.0,190286,67.5,4878,1.7,5609,...,7284,11.3,6086,9.5,4146,6.5,18943,29.5,8395,13.1
2,5001000US0102,0102,"Congressional District 2 (110th Congress), Ala...",281153,100.0,183535,65.3,6487,2.3,6565,...,7540,11.0,6036,8.8,4369,6.4,17616,25.7,10306,15.0
3,5001000US0103,0103,"Congressional District 3 (110th Congress), Ala...",286195,100.0,180623,63.1,5084,1.8,8632,...,7114,10.1,5686,8.0,4166,5.9,22127,31.3,10631,15.0
4,5001000US0104,0104,"Congressional District 4 (110th Congress), Ala...",282789,100.0,193779,68.5,2590,0.9,5829,...,5285,10.1,4803,9.2,2891,5.5,10959,20.9,10336,19.7
5,5001000US0105,0105,"Congressional District 5 (110th Congress), Ala...",278611,100.0,194472,69.8,5105,1.8,6607,...,8144,11.9,6459,9.5,4549,6.7,17484,25.6,7688,11.3
6,5001000US0106,0106,"Congressional District 6 (110th Congress), Ala...",271053,100.0,182817,67.4,7717,2.8,2096,...,6692,12.4,5131,9.5,3179,5.9,12319,22.9,6218,11.5
7,5001000US0107,0107,"Congressional District 7 (110th Congress), Ala...",282144,100.0,174760,61.9,6699,2.4,8089,...,9297,10.2,7224,7.9,6176,6.8,28901,31.8,11932,13.1
8,5001000US0200,0200,Congressional District (at Large) (110th Congr...,260978,100.0,152688,58.5,18340,7.0,14279,...,10141,12.3,8082,9.8,6095,7.4,20903,25.4,11534,14.0
9,5001000US0401,0401,"Congressional District 1 (110th Congress), Ari...",294076,100.0,179674,61.1,9273,3.2,5388,...,6721,10.7,5586,8.9,4143,6.6,17739,28.2,8408,13.4
